# Web srcapping computrabajo


In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Archivo CSV donde guardaremos los datos
archivo_csv = "C:\\Users\\John\\Desktop\\Proyectos\\Project_offers_jobs_env1\\data\\ofertas_computrabajo.csv"

# URL base para la búsqueda
base_url = "https://co.computrabajo.com/trabajo-de-analista-de-datos?p="
page = 1
ofertas = []

while True:
    url = base_url + str(page)
    print(f"Obteniendo enlaces de ofertas en {url}...")

    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})

    if response.status_code != 200:
        print(f"Error al obtener la página {page}: {response.status_code}")
        break  # Si hay un error (como 403 o 404), detenemos el scraping

    soup = BeautifulSoup(response.text, "html.parser")
    links = soup.find_all("a", class_="js-o-link")

    if not links:
        print("No se encontraron más ofertas. Fin del scraping.")
        break  # Si no hay más ofertas, salimos del bucle

    for link in links:
        job_url = "https://co.computrabajo.com" + link["href"]
        print(f"Extrayendo datos de: {job_url}")

        job_response = requests.get(job_url, headers={"User-Agent": "Mozilla/5.0"})
        if job_response.status_code != 200:
            print(f"Error al obtener oferta: {job_url}")
            continue

        job_soup = BeautifulSoup(job_response.text, "html.parser")

        titulo = job_soup.find("h1", class_="fwB fs24 mb5 box_detail w100_m").text.strip() if job_soup.find("h1", class_="fwB fs24 mb5 box_detail w100_m") else "No disponible"
        
        # Extraer empresa
        empresa = job_soup.find("a", class_="dIB fs16 js-o-link")
        empresa = empresa.text.strip() if empresa else "No disponible"

        # Extraer calificación de la empresa
        calificacion = job_soup.find("div", class_="mb5 mt5 fs16")
        calificacion = calificacion.text.strip().replace("★", "").strip() if calificacion else "No disponible"

        # Extraer ubicación (buscamos el último <p class="fs16"> que generalmente es la ubicación)
        ubicacion = "No disponible"
        p_tags = job_soup.find_all("p", class_="fs16")
        if p_tags:
            ubicacion = p_tags[-1].text.strip()  # Tomamos el último <p class="fs16">

        # Extraer salario
        salario = job_soup.find("span", class_="tag base mb10")
        salario = salario.text.strip() if salario else "No disponible"
        
        # Extraer tipo de contrato y jornada laboral
        contrato = "No disponible"
        jornada = "No disponible"
        tags = job_soup.find_all("span", class_="tag base mb10")
        if len(tags) > 1:
            contrato = tags[1].text.strip()
        if len(tags) > 2:
            jornada = tags[2].text.strip()

        # Extraer descripción
        descripcion = job_soup.find("p", class_="mbB")
        descripcion = descripcion.text.strip() if descripcion else "No disponible"

        # Extraer educación mínima, experiencia, edad y conocimientos
        educacion = "No disponible"
        experiencia = "No disponible"
        edad = "No disponible"
        conocimientos = "No disponible"

        requisitos = job_soup.find("ul", class_="disc mbB")
        if requisitos:
            items = requisitos.find_all("li")
            for item in items:
                texto = item.text.strip()
                if "Educación mínima" in texto:
                    educacion = texto.replace("Educación mínima: ", "")
                elif "experiencia" in texto:
                    experiencia = texto
                elif "Edad" in texto:
                    edad = texto.replace("Edad: ", "")
                elif "Conocimientos" in texto:
                    conocimientos = texto.replace("Conocimientos: ", "")

        ofertas.append({
            "title": titulo,
            "company": empresa,
            "rating": calificacion,
            "location": ubicacion,
            "salary": salario,
            "contract": contrato,
            "schedule": jornada,
            "description": descripcion,
            "education": educacion,
            "experience": experiencia,
            "age": edad,
            "skills": conocimientos,
            "url": job_url
        })

        time.sleep(1)  # Pausa para evitar bloqueos

    page += 1  # Pasar a la siguiente página

# Guardar datos en CSV
with open(archivo_csv, "w", newline="", encoding="utf-8") as file:
    fieldnames = [
        "title", "company", "rating", "location", "salary", "contract", "schedule",
        "description", "education", "experience", "age", "skills", "url"
    ]
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()  # Escribir la cabecera del CSV
    writer.writerows(ofertas)  # Escribir los datos

print(f"Scraping completado. Se guardaron {len(ofertas)} ofertas en {archivo_csv}.")


Obteniendo enlaces de ofertas en https://co.computrabajo.com/trabajo-de-analista-de-datos?p=1...
Extrayendo datos de: https://co.computrabajo.com/ofertas-de-trabajo/oferta-de-trabajo-de-analista-de-programacion-y-datos-medellin-en-medellin-EC1E91FE77BC104561373E686DCF3405#lc=ListOffers-Score4-0
Extrayendo datos de: https://co.computrabajo.com/ofertas-de-trabajo/oferta-de-trabajo-de-analista-operaciones-analista-de-datos-en-bello-679C187A2846861061373E686DCF3405#lc=ListOffers-Score4-1
Extrayendo datos de: https://co.computrabajo.com/ofertas-de-trabajo/oferta-de-trabajo-de-analista-de-datos-analista-bi-manejo-de-ingles-b2-o-c1-dominio-de-excel-y-power-bi-avanzado-en-medellin-B74E936EEB8DA69761373E686DCF3405#lc=ListOffers-Score4-2
Extrayendo datos de: https://co.computrabajo.com/ofertas-de-trabajo/oferta-de-trabajo-de-analista-de-datos-en-bogota-dc-B9003DF77000C59661373E686DCF3405#lc=ListOffers-Score4-3
Extrayendo datos de: https://co.computrabajo.com/ofertas-de-trabajo/oferta-de-trabajo-